In [1]:
!pip install Faker
!pip install mysql-connector-python

  Using cached mysql_connector_python-9.4.0-cp310-cp310-manylinux_2_28_x86_64.whl.metadata (7.3 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 33.9/33.9 MB 371.6 kB/s  0:00:58m0:00:0100:03


In [2]:
import random
from faker import Faker
from datetime import datetime, timedelta
import uuid

In [ ]:
def generate_sample_data():
    fake = Faker("es_ES")

    # Generate 50 fake users
    users = []
    for i in range(0, 50):
        user_id = fake.uuid4()
        users.append(
            {
                "user_id": user_id,
                "nombre": fake.name(),
                "email": fake.email(),
                "telefono": fake.phone_number(),
                "direccion": fake.address(),
                "fecha_registro": (
                    datetime.now() - timedelta(days=random.randint(180, 365))
                ).strftime("%Y-%m-%d %H:%M:%S"),
            }
        )
    print("--- 50 Sample Users (Salvadoran Spanish) ---")
    print(users[:10])

    # Generate 6 months of billing data for each user
    billing_data = []
    service = "Internet Residencial"
    base_amount = 35.00
    today = datetime.now()

    for user in users:
        for month_offset in range(6, 0, -1):
            bill_date = today - timedelta(days=30 * month_offset)
            amount = base_amount + round(random.uniform(-5.00, 5.00), 2)
            status = random.choice(["Cancelado", "Pendiente"])

            billing_id = fake.uuid4()
            billing_data.append(
                {
                    "billing_id": billing_id,
                    "user_id": user["user_id"],
                    "fecha_factura": bill_date.strftime("%Y-%m-%d 00:00:00"),
                    "monto": amount,
                    "estado": status,
                    "servicio": service,
                }
            )
    print("\n--- 6 Months of Billing Data for 'Internet Residencial' ---")
    print(billing_data[:10])
    return users, billing_data

In [4]:
users, billing_data = generate_sample_data()

--- 50 Sample Users (Salvadoran Spanish) ---
[{'user_id': '8c78dec4-19e1-49b1-9d64-1d20600f40d3', 'nombre': 'Amarilis de Lledó', 'email': 'monica16@example.net', 'telefono': '+34832 376 422', 'direccion': 'Camino de Régulo Izquierdo 30 Puerta 8 \nCuenca, 09231', 'fecha_registro': '2024-10-22 20:45:05'}, {'user_id': '451f8043-a4ca-4dae-a31f-4dc06dfdc27c', 'nombre': 'Antonia Real Garay', 'email': 'lelias@example.org', 'telefono': '+34952 044 659', 'direccion': 'Vial Daniela Bejarano 635\nSevilla, 06491', 'fecha_registro': '2025-01-31 20:45:05'}, {'user_id': 'e6b11921-722c-4cb1-af33-d3e06b15eaf8', 'nombre': 'Melisa Palomo-Lago', 'email': 'francisco14@example.org', 'telefono': '+34 828 86 09 84', 'direccion': 'Camino Íngrid Pomares 12 Piso 7 \nTarragona, 26155', 'fecha_registro': '2025-03-15 20:45:05'}, {'user_id': 'b41b8cb8-7155-4837-98bf-8510bbe87636', 'nombre': 'Paulina Vicente Amigó', 'email': 'amadorjudith@example.org', 'telefono': '+34 941529462', 'direccion': 'C. de Fabián Castelló 

In [ ]:
# Parámetros de conexión MYSQL
config = {
    "user": "icc115",
    "password": "icc115",
    "host": "127.0.0.1",
    "database": "icc115",
}

In [ ]:
import mysql.connector
from mysql.connector import Error


def insert_multiple_records(records, sql_query):
    """Inserta múltiples registros en la tabla 'usuarios'."""
    try:
        # Establecer la conexión con la base de datos
        conn = mysql.connector.connect(**config)

        if conn.is_connected():
            cursor = conn.cursor()

            # Ejecutar la inserción masiva
            cursor.executemany(sql_query, records)

            # Confirmar los cambios (obligatorio para guardar los datos)
            conn.commit()

            print(f"{cursor.rowcount} registros insertados correctamente.")

    except Error as e:
        print(f"Error al insertar múltiples registros: {e}")

    finally:
        # Cerrar el cursor y la conexión
        if "conn" in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print("Conexión a MySQL cerrada.")


def convertir_dict_a_tupla(lista_de_diccionarios, orden_columnas):
    """
    Convierte una lista de diccionarios a una lista de tuplas,
    garantizando el orden de los valores.

    Args:
        lista_de_diccionarios (list): La lista de diccionarios a convertir.
        orden_columnas (list): Una lista de strings con las claves en el orden deseado.

    Returns:
        list: Una lista de tuplas con los valores ordenados.
    """
    records_as_tuples = []
    for record in lista_de_diccionarios:
        try:
            # Obtener los valores en el orden especificado
            tuple_record = tuple(record[key] for key in orden_columnas)
            records_as_tuples.append(tuple_record)
        except KeyError as e:
            print(
                f"Advertencia: El diccionario {record} no contiene la clave {e}. Se omite."
            )
    return records_as_tuples

In [ ]:
# Insertando usuarios
insert_users_sql_query = "INSERT INTO users (user_id, nombre, email, telefono, direccion, fecha_registro) VALUES (UUID_TO_BIN(%s), %s, %s, %s, %s, %s)"
tuplas_users = convertir_dict_a_tupla(
    users, ["user_id", "nombre", "email", "telefono", "direccion", "fecha_registro"]
)
print(tuplas_users[:10])
# insert_multiple_records(tuplas_users, insert_users_sql_query)

[('8c78dec4-19e1-49b1-9d64-1d20600f40d3', 'Amarilis de Lledó', 'monica16@example.net', '+34832 376 422', 'Camino de Régulo Izquierdo 30 Puerta 8 \nCuenca, 09231', '2024-10-22 20:45:05'), ('451f8043-a4ca-4dae-a31f-4dc06dfdc27c', 'Antonia Real Garay', 'lelias@example.org', '+34952 044 659', 'Vial Daniela Bejarano 635\nSevilla, 06491', '2025-01-31 20:45:05'), ('e6b11921-722c-4cb1-af33-d3e06b15eaf8', 'Melisa Palomo-Lago', 'francisco14@example.org', '+34 828 86 09 84', 'Camino Íngrid Pomares 12 Piso 7 \nTarragona, 26155', '2025-03-15 20:45:05'), ('b41b8cb8-7155-4837-98bf-8510bbe87636', 'Paulina Vicente Amigó', 'amadorjudith@example.org', '+34 941529462', 'C. de Fabián Castelló 93\nAlicante, 10679', '2024-10-06 20:45:05'), ('657f375f-ca4f-4494-bace-76e1519a4952', 'Josep Casemiro Tamarit Criado', 'aragoneseugenio@example.com', '+34978998137', 'Glorieta Isidora Borrego 60\nValladolid, 30571', '2025-01-18 20:45:05'), ('d9c14bc4-1d85-4abf-84b2-29599427dbbd', 'Joel de Alemán', 'loidablanca@exampl

In [ ]:
# Insertando facturas
insert_billing_data_sql_query = "INSERT INTO billing_data (billing_id, user_id, fecha_factura, monto, estado, servicio) VALUES (UUID_TO_BIN(%s), UUID_TO_BIN(%s), %s, %s, %s, %s)"
tuplas_billing_data = convertir_dict_a_tupla(
    billing_data,
    ["billing_id", "user_id", "fecha_factura", "monto", "estado", "servicio"],
)
print(tuplas_billing_data[:10])
# insert_multiple_records(tuplas_billing_data, insert_billing_data_sql_query)

[('f81a6dda-e5a7-42e5-94de-8843f3842d1a', '8c78dec4-19e1-49b1-9d64-1d20600f40d3', '2025-04-09 00:00:00', 36.06, 'Pendiente', 'Internet Residencial'), ('4d68716f-15a4-4513-aeec-a42cd1b8c944', '8c78dec4-19e1-49b1-9d64-1d20600f40d3', '2025-05-09 00:00:00', 30.54, 'Pendiente', 'Internet Residencial'), ('d93c0b49-9b0e-4ab4-942d-c62e5cca5bc7', '8c78dec4-19e1-49b1-9d64-1d20600f40d3', '2025-06-08 00:00:00', 35.51, 'Cancelado', 'Internet Residencial'), ('89c4d60c-87d8-4efe-b6d6-4eac08d99c18', '8c78dec4-19e1-49b1-9d64-1d20600f40d3', '2025-07-08 00:00:00', 37.42, 'Pendiente', 'Internet Residencial'), ('79831f84-d25c-4218-967f-bdef6404f5ba', '8c78dec4-19e1-49b1-9d64-1d20600f40d3', '2025-08-07 00:00:00', 33.22, 'Pendiente', 'Internet Residencial'), ('139caa73-d76c-4e3d-ab36-b52213417647', '8c78dec4-19e1-49b1-9d64-1d20600f40d3', '2025-09-06 00:00:00', 37.33, 'Cancelado', 'Internet Residencial'), ('a958219b-d6fd-4437-be36-0b130a0ffed4', '451f8043-a4ca-4dae-a31f-4dc06dfdc27c', '2025-04-09 00:00:00', 3

In [ ]:
import json


def cargar_json(file_path):
    try:
        with open(file_path, "r", encoding="utf-8") as file:
            data = json.load(file)
            print("Datos cargados desde el archivo:")
            print(data[:1])
            print("El tipo de dato cargado es:", type(data))
            return data
    except FileNotFoundError:
        print(f"Error: El archivo '{file_path}' no fue encontrado.")
    except json.JSONDecodeError:
        print(
            "Error: No se pudo decodificar el JSON. Asegúrate de que el formato sea correcto."
        )

### Cargando conversaciones

In [ ]:
conversations = []
new_client_sample_conversations_file_path = "./new_client_sample_conversations.json"
new_client_sample_conversations = cargar_json(new_client_sample_conversations_file_path)
conversations.extend(new_client_sample_conversations)

billing_sample_conversations_file_path = "./billing_sample_conversations.json"
billing_sample_conversations = cargar_json(billing_sample_conversations_file_path)
conversations.extend(billing_sample_conversations)

conexion_issues_conversations_file_path = "./conexion_issues_conversations.json"
conexion_issues_conversations = cargar_json(conexion_issues_conversations_file_path)
conversations.extend(conexion_issues_conversations)

print(conversations[:2])

Datos cargados desde el archivo:
[{'cliente': {'nombre_completo': 'Laura Patricia Vásquez Orellana', 'telefono': '7987-6543'}, 'satisfaccion': 5, 'historial': [{'sender': 'cliente', 'text': 'Hola, estoy interesada en contratar un plan de internet residencial. ¿Qué opciones tienen?'}, {'sender': 'agente', 'text': '¡Buenos días, Sra. Vásquez! Tenemos varios planes residenciales con diferentes velocidades. Para poderle dar la mejor recomendación, ¿podría indicarme su DUI para verificar la cobertura en su área?'}, {'sender': 'cliente', 'text': 'Sí, claro. Mi DUI es 09876543-2. Vivo en la Colonia Escalón.'}, {'sender': 'agente', 'text': 'Gracias. He verificado y su zona cuenta con cobertura de fibra óptica. Le recomiendo nuestro plan de 100 Mbps, que incluye una instalación gratuita por promoción. ¿Le gustaría que le brinde más detalles?'}, {'sender': 'cliente', 'text': 'Me parece muy bien. Sí, por favor, envíeme la información completa al correo electrónico.'}, {'sender': 'agente', 'text':

In [14]:
def load_users_from_mysql():
    try:
        conn = mysql.connector.connect(**config)
        if conn.is_connected():
            cursor = conn.cursor(dictionary=True)
            cursor.execute(
                "SELECT BIN_TO_UUID(user_id) as user_id, nombre, email, telefono, direccion, fecha_registro FROM users"
            )
            users_from_db = cursor.fetchall()
            print("Usuarios cargados desde MySQL:")
            print(users_from_db[:5])
            return users_from_db
    except Error as e:
        print(f"Error al cargar usuarios: {e}")
    finally:
        if "conn" in locals() and conn.is_connected():
            cursor.close()
            conn.close()


users_db = load_users_from_mysql()

Usuarios cargados desde MySQL:
[{'user_id': '006590d0-ecb3-430c-a51d-06716d9400a9', 'nombre': 'Basilio Vargas Manuel', 'email': 'inocenciorozas@example.net', 'telefono': '+34946 11 91 97', 'direccion': 'Acceso Rafa Luna 91\nSegovia, 17869', 'fecha_registro': datetime.datetime(2025, 1, 20, 18, 32, 49)}, {'user_id': '0065bdac-ede1-4042-9230-90b6eea4de0d', 'nombre': 'Mauricio Hernandez', 'email': 'ciriacozaragoza@example.org', 'telefono': '+34 820432511', 'direccion': 'Alameda de Merche Ortiz 54\nTarragona, 11032', 'fecha_registro': datetime.datetime(2024, 10, 8, 18, 32, 49)}, {'user_id': '01b3aaec-a50d-4f22-be6d-e87b98c8a4a6', 'nombre': 'Bárbara Sotelo Aller', 'email': 'fernandasobrino@example.com', 'telefono': '+34 888 064 938', 'direccion': 'Camino Saturnina Canals 3 Piso 8 \nPalencia, 46725', 'fecha_registro': datetime.datetime(2025, 2, 3, 18, 32, 49)}, {'user_id': '06875aed-2412-4276-a75a-731586f82ce6', 'nombre': 'María Del Carmen Tudela Guardia', 'email': 'qsedano@example.net', 'tel

In [50]:
import random

conversations_data = []

for conversation in conversations:
    # Generar un número entero aleatorio entre 1 y 10 (inclusive)
    random_int = random.randint(0, 49)
    ramdom_user = users_db[random_int]
    session_id = str(uuid.uuid4())
    random_days_ago = random.randint(0, 180)
    random_date = datetime.now() - timedelta(days=random_days_ago)

    start_time = random_date  # Timestamp inicial de la conversación

    conversation_data = {}
    conversation_data["session_id"] = session_id
    conversation_data["user_id"] = ramdom_user["user_id"]
    # Array para almacenar los mensajes de la conversación
    conversation_mensajes = []

    # Iterar sobre los mensajes de la conversación
    for i, message in enumerate(conversation["historial"]):
        random_minutes = random.randint(0, 15)
        current_time = start_time + timedelta(minutes=random_minutes)
        message_time = current_time.strftime("%Y-%m-%d %H:%M:%S")
        message_id = str(uuid.uuid4())

        message_data = {
            "session_id": session_id,
            "message_id": message_id,
            "timestamp": message_time,
            "sender": message["sender"],
            "text": message["text"],
        }

        # Agregar fecha y hora de inicio y fin de la conversación
        if i == 0:
            conversation_data["fecha_inicio"] = message_time
        if i == len(conversation["historial"]) - 1:
            conversation_data["fecha_fin"] = message_time

        # Se agrega el mensaje al array de mensajes de la conversación
        conversation_mensajes.append(message_data)

        start_time = current_time  # Actualizar start_time para el próximo mensaje

    # Se agrega el array de mensajes a la conversación
    conversation_data["messages"] = conversation_mensajes

    # Agregando support tickets
    if "support_ticket" in conversation:
        # Generar un support ticket para la conversación

        ticket_id = str(uuid.uuid4())
        ticket_created_at = conversation_data["fecha_fin"]
        estado = conversation["support_ticket"]["estado"]
        problema = conversation["support_ticket"]["problema"]
        descripcion = conversation["support_ticket"]["descripcion"]
        prioridad = conversation["support_ticket"]["prioridad"]

        support_ticket_random_hours = random.randint(
            0, 24 * 3
        )  # Hasta 3 días después de la creación
        support_ticket_fecha_cierre = start_time + timedelta(
            hours=support_ticket_random_hours
        )

        fecha_cierre = support_ticket_fecha_cierre if estado == "Cerrado" else None

        support_ticket = {
            "ticket_id": ticket_id,
            "user_id": conversation_data["user_id"],
            "prioridad": prioridad,
            "estado": estado,
            "problema": problema,
            "descripcion": descripcion,
            "fecha_creacion": conversation_data["fecha_fin"],
            "fecha_cierre": fecha_cierre,
        }

        conversation_data["support_ticket"] = support_ticket

    # Agregar feedback
    feedback_id = str(uuid.uuid4())
    if "feedback" in conversation:
        # Generar un feedback para la conversación
        rating = conversation["feedback"]["rating"]
        comentarios = conversation["feedback"]["comentarios"]
    else:
        rating = random.randint(1, 5)
        comentarios = random.choice(
            [
                "Excelente atención.",
                "Resolvieron mi problema rápidamente.",
                "El servicio podría mejorar.",
                "No quedé satisfecho con la respuesta.",
                "Muy buen soporte técnico.",
                "La conversación fue clara y útil.",
                "Me gustaría una atención más rápida.",
                "El agente fue muy amable.",
                "No entendí la solución propuesta.",
                "Todo perfecto, gracias.",
            ]
        )

    fecha_feedback = conversation_data["fecha_fin"]

    feedback = {
        "feedback_id": feedback_id,
        "session_id": session_id,
        "user_id": conversation_data["user_id"],
        "rating": rating,
        "comentarios": comentarios,
        "fecha_feedback": fecha_feedback,
    }

    conversation_data["feedback"] = feedback
    conversation_data["satisfaccion"] = (
        conversation["satisfaccion"] if "satisfaccion" in conversation else rating
    )

    # Agregar visita técnica
    if "visita_tecnica" in conversation:
        visita_id = str(uuid.uuid4())
        tecnico = conversation["visita_tecnica"]["tecnico"]
        fecha_visita = conversation["visita_tecnica"]["fecha_visita"]

        visita_tecnica_estado_choices = [
            # "Programada",
            # "En Curso",
            "Completada",
            "Cancelada",
            "Cerrada",
        ]
        visita_tecnica_estado = random.choice(visita_tecnica_estado_choices)

        visita_tecnica = {
            "visit_id": visita_id,
            "ticket_id": ticket_id if "support_ticket" in conversation else None,
            "user_id": conversation_data["user_id"],
            "tecnico": tecnico,
            "fecha_visita": fecha_visita,
            "estado": visita_tecnica_estado,
        }

        conversation_data["visita_tecnica"] = visita_tecnica

    conversations_data.append(conversation_data)

print(conversations_data[10])

{'session_id': 'a3833464-1dd7-47b7-b1f6-e8c1ff4839d4', 'user_id': '0065bdac-ede1-4042-9230-90b6eea4de0d', 'fecha_inicio': '2025-06-26 22:11:17', 'fecha_fin': '2025-06-26 22:37:17', 'messages': [{'session_id': 'a3833464-1dd7-47b7-b1f6-e8c1ff4839d4', 'message_id': 'de85cbe2-ac7d-4e19-984b-0c6c15d03039', 'timestamp': '2025-06-26 22:11:17', 'sender': 'cliente', 'text': 'Hola, mi internet está fallando de forma intermitente desde hace dos días. Se desconecta y reconecta solo.'}, {'session_id': 'a3833464-1dd7-47b7-b1f6-e8c1ff4839d4', 'message_id': 'ac10f76b-310a-4162-b4c5-0cc657211544', 'timestamp': '2025-06-26 22:20:17', 'sender': 'agente', 'text': 'Buenas tardes, Sr. Solís. Lamento el inconveniente. Para iniciar, ¿podría, por favor, reinicializar su router? Apáguelo, espere 30 segundos y vuelva a encenderlo.'}, {'session_id': 'a3833464-1dd7-47b7-b1f6-e8c1ff4839d4', 'message_id': '418d189e-cdba-4f4c-aab4-22e0d331dfaa', 'timestamp': '2025-06-26 22:33:17', 'sender': 'cliente', 'text': 'Ya lo 

In [56]:
messages = []
feedbacks = []
technical_visits = []
support_tickets = []
conversation_records = []
for conv in conversations_data:
    messages_tuples = [
        (
            msg["session_id"],
            msg["message_id"],
            msg["timestamp"],
            msg["sender"],
            msg["text"],
        )
        for msg in conv["messages"]
    ]
    messages.extend(messages_tuples)

    
    feedbacks_tuples = [
        (
            conv["feedback"]["feedback_id"],
            conv["feedback"]["session_id"],
            conv["feedback"]["user_id"],
            conv["feedback"]["rating"],
            conv["feedback"]["comentarios"],
            conv["feedback"]["fecha_feedback"],
        )
    ]
    feedbacks.extend(feedbacks_tuples)

    if "visita_tecnica" in conv:
        visita = conv["visita_tecnica"]
        technical_visits.append(
            (
                visita["visit_id"],
                visita["ticket_id"],
                visita["user_id"],
                visita["tecnico"],
                visita["fecha_visita"],
                visita["estado"],
            )
        )
    if "support_ticket" in conv:
        support_tickets.append(
            (
                conv["support_ticket"]["ticket_id"],
                conv["support_ticket"]["user_id"],
                conv["support_ticket"]["prioridad"],
                conv["support_ticket"]["estado"],
                conv["support_ticket"]["problema"],
                conv["support_ticket"]["descripcion"],
                conv["support_ticket"]["fecha_creacion"],
                conv["support_ticket"]["fecha_cierre"],
            )
        )
    conversation_records.append(
        (
            conv["session_id"],
            conv["user_id"],
            conv["fecha_inicio"],
            conv["fecha_fin"],
            conv["satisfaccion"],
        )
    )

In [54]:
# Insertando facturas
conversations_data_sql_query = "INSERT INTO conversations (session_id, user_id, fecha_inicio, fecha_fin, satisfaccion) VALUES (UUID_TO_BIN(%s), UUID_TO_BIN(%s), %s, %s, %s)"
print(conversation_records[:10])
insert_multiple_records(conversation_records, conversations_data_sql_query)

[('a4db5689-8b68-4f7a-b2f8-9aef82bffd58', '0065bdac-ede1-4042-9230-90b6eea4de0d', '2025-09-11 22:17:17', '2025-09-11 22:55:17', 5), ('4107a4c2-2435-42f0-9134-e9253e3c9d6d', '565aedf2-1cf0-4bb2-be99-d63a1aeecac6', '2025-05-16 22:08:17', '2025-05-16 22:49:17', 2), ('acfd7bf9-00e3-46b6-9433-a1bc5138647a', '848390bf-fd7d-4fe5-b355-f814a826a5b1', '2025-06-03 22:07:17', '2025-06-03 23:00:17', 4), ('ed42a796-cecd-40a3-8839-1966a57c7959', '43f51d2a-fda4-4855-9e1d-c0d2f81f4d91', '2025-05-28 22:09:17', '2025-05-28 22:48:17', 1), ('3ec28d3b-2be5-4f4b-b32f-ccaa212220e2', '4d1ac377-97cb-475d-8e7b-001ef7f68a5a', '2025-08-05 22:15:17', '2025-08-05 22:57:17', 5), ('c044c4dd-7298-49dd-9622-f610fe1879bd', 'c8f42192-2603-40c8-ade0-5e37e999c05c', '2025-05-18 22:02:17', '2025-05-18 22:32:17', 5), ('0483cc8d-f5dc-48cd-b157-13665688aac7', 'e99b2054-53e9-4d99-9fd3-327a2c53ead5', '2025-06-22 22:09:17', '2025-06-22 22:57:17', 2), ('0e25fbfd-683d-4b46-ac65-12771d935336', '6e55da9d-0a40-4db7-ab9c-44bcbc2cb20e', '

In [59]:
messages_sql_query = "INSERT INTO messages (session_id, message_id, timestamp, sender, text) VALUES (UUID_TO_BIN(%s), UUID_TO_BIN(%s), %s, %s, %s)"
insert_multiple_records(messages, messages_sql_query)

84 registros insertados correctamente.
Conexión a MySQL cerrada.


In [61]:
feedbacks_sql_query = "INSERT INTO feedback (feedback_id, session_id, user_id, rating, comentarios, fecha_feedback) VALUES (UUID_TO_BIN(%s), UUID_TO_BIN(%s), UUID_TO_BIN(%s), %s, %s, %s)"
insert_multiple_records(feedbacks, feedbacks_sql_query)

15 registros insertados correctamente.
Conexión a MySQL cerrada.


In [65]:
support_tickets_sql_query = "INSERT INTO support_tickets (ticket_id, user_id, prioridad, estado, problema, descripcion, fecha_creacion, fecha_cierre) VALUES (UUID_TO_BIN(%s), UUID_TO_BIN(%s), %s, %s, %s, %s, %s, %s)"
insert_multiple_records(support_tickets, support_tickets_sql_query)

7 registros insertados correctamente.
Conexión a MySQL cerrada.


In [66]:
technical_visits_sql_query = "INSERT INTO technical_visits (visit_id, ticket_id, user_id, tecnico, fecha_visita, estado) VALUES (UUID_TO_BIN(%s), UUID_TO_BIN(%s), UUID_TO_BIN(%s), %s, %s, %s)"
insert_multiple_records(technical_visits, technical_visits_sql_query)

3 registros insertados correctamente.
Conexión a MySQL cerrada.
